<a href="https://colab.research.google.com/github/gogela/Colab-D3js/blob/main/Traffic_chord_diagram_d3js.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visualize traffic using D3.js chord diagram

In [ ]:
#mockup IP data in pd df

from random import randint
import pandas as pd
import numpy as np

maxip=50
maxpkt=100000
maxflows=100
iplist=['10.{}.{}.{}'.format(randint(0,254),randint(0,254),randint(1,254)) for x in range(maxip)]
flows = [[iplist[randint(0,maxip-1)],iplist[randint(0,maxip-1)],randint(1,maxpkt)] for x in range(maxflows)]
df = pd.DataFrame(np.array(flows), columns=['src', 'dst', 'pkt'])

#convert to matrix for d3
indexes = list(df.src.unique())
indexes += list(df.dst.unique())
indexes = sorted(list(set(indexes)))
l = len(indexes)
matrix = [[0 for i in range(l)] for j in range(l)]

def update_matrix(x):
  i=indexes.index(x.src)
  j=indexes.index(x.dst)
  matrix[i][j]+=int(x.pkt)
  matrix[j][i]+=int(x.pkt)
df.apply(lambda x: update_matrix(x), axis=1)


In [ ]:
#data to string for HTML injection
datajs = str(matrix)
labels = str(indexes)
size=1400
radius=400

In [ ]:
#d3js html - parametrized
html=f'''
<!DOCTYPE html>
<meta charset="utf-8">

<!-- Load d3.js -->
<script src="https://d3js.org/d3.v4.js"></script>

<!-- Create a div where the graph will take place -->
<div id="my_dataviz"></div>
<script>

// create the svg area
var svg = d3.select("#my_dataviz")
  .append("svg")
    .attr("width", {size})
    .attr("height", {size})
  .append("g")
    .attr("transform", "translate({size/2},{size/2})")

// create input data: a square matrix that provides flow between entities
var matrix = {datajs};
var labels = {labels}


// give this matrix to d3.chord(): it will calculates all the info we need to draw arc and ribbon
var res = d3.chord()
    .padAngle(0.05)     // padding between entities (black arc)
    .sortSubgroups(d3.descending)
    (matrix)

//index based color
var myColor = d3.scaleSequential().domain([1,res.length]).interpolator(d3.interpolateViridis);

// add the groups on the inner part of the circle
svg
  .datum(res)
  .append("g")
  .selectAll("g")
  .data(function(d) {{  return d.groups; }})
  .enter()
  .append("g")
  .append("path")
    .style("fill", "red")
    .style("stroke", "black")
    .attr("d", d3.arc()
      .innerRadius({radius})
      .outerRadius({radius+10})
    )

// Add the links between groups
svg
  .datum(res)
  .append("g")
  .selectAll("path")
  .data(function(d) {{ return d; }})
  .enter()
  .append("path")
    .attr("d", d3.ribbon()
      .radius({radius})
    )
    .style("fill", function(d,i){{ return myColor(i) }})
    .style("stroke", "black");

var offset =5
svg
.selectAll(".group-tick-label")
  .data(function(d) {{  return d.groups; }})
  .enter()
  //.filter(function(d) {{ return d.value % 25 === 0; }})
  .append("g")
    .attr("transform", function(d) {{ var x = "rotate(" + ((d.startAngle+(d.endAngle-d.startAngle)/2) * 180 / Math.PI - 90) + ") translate(" + ({radius} + offset) + ",0)";  return x }})
  .append("text")
    .attr("x", 8)
    .attr("dy", ".35em")
    .attr("transform", function(d) {{  return d.startAngle > Math.PI ? "rotate(180) translate(-16)" : null; }})
    .style("text-anchor", function(d) {{ return d.startAngle > Math.PI ? "end" : null; }})
    .text(function(d,i) {{ return labels[i] }})
    .style("font-size", 12)


</script>
'''

In [ ]:
#
from IPython.core.display import HTML
HTML(html)

In [ ]:
f=open('diagram.html','w')
f.write(html)
f.close()